In [1]:
import importlib

In [2]:
import hnsw
importlib.reload(hnsw)
from hnsw import HNSW
from hnsw import heuristic
import numpy as np
from datasets import load_sift_dataset, calculate_recall

In [9]:
import merge_hnsw
importlib.reload(merge_hnsw)
from merge_hnsw import hnsw_general_merge
from merge_hnsw import merge_naive

In [5]:
distance_count = 0
def l2_distance(a, b):
    global distance_count
    distance_count+=1
    return np.linalg.norm(a - b)

In [6]:
hnsw_a = HNSW( distance_func=l2_distance, m=5, m0=7, ef=10, ef_construction=30,  neighborhood_construction = heuristic)
hnsw_b = HNSW( distance_func=l2_distance, m=5, m0=7, ef=10, ef_construction=30,  neighborhood_construction = heuristic)

In [7]:
hnsw_a.load('save/sift1m/hnsw_a.txt')
hnsw_b.load('save/sift1m/hnsw_b.txt')

In [8]:
merged_data = hnsw_a.data.copy()
merged_data.update(hnsw_b.data)

In [15]:
merge_ef = 20
def layer_merge_naive_func(hnsw_a, hnsw_b, merged_data, level):
  return merge_naive(hnsw_a, hnsw_b, merged_data, level, search_ef=merge_ef)

In [12]:
%%time
distance_count = 0
hnsw_merged_naive = hnsw_general_merge(hnsw_a, hnsw_b, merged_data, layer_merge_naive_func)
print(f'Numer of distance calculated during the merge: {distance_count}') # 696869746

Merging level: 0


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500000/500000 [8:31:10<00:00, 16.30it/s]


Merging level: 1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31358/31358 [09:36<00:00, 54.36it/s]


Merging level: 2


...

Merging level: 3


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 124/124 [00:00<00:00, 310.28it/s]


Merging level: 4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 2042.85it/s]

Total metric calculation: 8683010381
CPU times: user 10h 50min 7s, sys: 1h 42min 55s, total: 12h 33min 3s
Wall time: 14h 20min 39s


In [16]:
hnsw_merged_naive.save(f'save/sift1m/hnsw_merged_naive_ef{merge_ef}.txt')

In [13]:
print(f'Numer of distance calculated during the merge: {distance_count}') # 8683010381

Numer of distance calculated during the merge: 8683010381


In [17]:
_, test_data, groundtruth_data = load_sift_dataset(train_file = None,
                                                      test_file='../datasets/sift1m-128d/sift_query.fvecs',
                                                      groundtruth_file='../datasets/sift1m-128d/sift_groundtruth.ivecs')

In [22]:
search_ef = 32
distance_count = 0
recall, _ = calculate_recall(hnsw_merged_naive, test_data, groundtruth=groundtruth_data, k=5, ef=search_ef)
print(f'{search_ef},{recall}, {distance_count/len(test_data) }') 

Calculating recall...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:38<00:00, 102.02it/s]

32,0.8929600000000001, 772.3489


In [23]:
search_ef = 40
distance_count = 0
recall, _ = calculate_recall(hnsw_merged_naive, test_data, groundtruth=groundtruth_data, k=5, ef=search_ef)
print(f'{search_ef},{recall}, {distance_count/len(test_data) }') 

Calculating recall...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:59<00:00, 83.67it/s]

40,0.9169200000000001, 896.6725


In [24]:
search_ef = 64
distance_count = 0
recall, _ = calculate_recall(hnsw_merged_naive, test_data, groundtruth=groundtruth_data, k=5, ef=search_ef)
print(f'{search_ef},{recall}, {distance_count/len(test_data) }') 

Calculating recall...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [24:46<00:00,  6.73it/s]

64,0.95306, 1256.9268
